Objectives
 

Project Brief
You work for Spark Funds, an asset management company. Spark Funds wants to make investments in a few companies. The CEO of Spark Funds wants to understand the global trends in investments so that she can take the investment decisions effectively.

 

Business and Data Understanding
Spark Funds has two minor constraints for investments:

It wants to invest between 5 to 15 million USD per round of investment

It wants to invest only in English-speaking countries because of the ease of communication with the companies it would invest in

For your analysis, consider a country to be English speaking only if English is one of the official languages in that country

You may use this list: Click here for a list of countries where English is an official language.

 

These conditions will give you sufficient information for your initial analysis. Before getting to specific questions, let’s understand the problem and the data first.

 

1. What is the strategy?

Spark Funds wants to invest where most other investors are investing. This pattern is often observed among early stage startup investors.

 

2. Where did we get the data from? 

We have taken real investment data from crunchbase.com, so the insights you get may be incredibly useful. For this group project, we have divided the data into the following files:

 

You have to use three main data tables for the entire analysis (available for download on the next page):

 

3. What is Spark Funds’ business objective?

The business objectives and goals of data analysis are pretty straightforward.

Business objective: The objective is to identify the best sectors, countries, and a suitable investment type for making investments. The overall strategy is to invest where others are investing, implying that the 'best' sectors and countries are the ones 'where most investors are investing'.
Goals of data analysis: Your goals are divided into three sub-goals:
Investment type analysis: Comparing the typical investment amounts in the venture, seed, angel, private equity etc. so that Spark Funds can choose the type that is best suited for their strategy.
Country analysis: Identifying the countries which have been the most heavily invested in the past. These will be Spark Funds’ favourites as well.
Sector analysis: Understanding the distribution of investments across the eight main sectors. (Note that we are interested in the eight 'main sectors' provided in the mapping file. The two files — companies and rounds2 — have numerous sub-sector names; hence, you will need to map each sub-sector to its main sector.)
 

4. How do you approach the case study? What are the deliverables?

The entire case study is divided into checkpoints to help you navigate. For each checkpoint, you are advised to fill in the tables into the spreadsheet provided in the download segment. The tables are also mentioned under the 'Results Expected' section after each checkpoint. Since this is the first case study, you have been provided with some additional guidance. Going forward you will be expected to structure and solve the problem by yourself, just like you would be solving problems in real life scenarios.

 

Important Note: All your code has to be submitted in one Jupyter notebook. For every checkpoint, keep writing code in one well-commented Jupyter notebook which you can submit at the end.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load company dataset
# Without encoding the read_csv was giving an error "Unicode Decode Error"
companies_df = pd.read_csv('Data/companies.txt',delimiter='\t',encoding='unicode_escape')

In [4]:
companies_df.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [3]:
# Load funding rounds file, got same error on the Unicode/Decode Error, used recommended encoding.
rounds2_df = pd.read_csv('Data/rounds2.csv',encoding='unicode_escape')

In [4]:
# Quickly inspect the data frame
rounds2_df.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [7]:
rounds2_df.count()

company_permalink          114949
funding_round_permalink    114949
funding_round_type         114949
funding_round_code          31140
funded_at                  114949
raised_amount_usd           94959
dtype: int64

In [8]:
# Load the mapping file for the sector classification
sector_mapping_df = pd.read_csv('Data/mapping.csv')

In [9]:
sector_mapping_df.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [10]:
# Load the country codes file
# source: https://github.com/datasets/country-codes/blob/master/data/country-codes.csv
# columns of interest: country name: official_name_en, Country code: ISO3166-1-Alpha-3
country_df = pd.read_csv('Data/country-codes.csv')

In [11]:
country_df.head()

,FIFA,Dial,ISO3166-1-Alpha-3,MARC,is_independent,ISO3166-1-numeric,GAUL,FIPS,WMO,ISO3166-1-Alpha-2,...,Sub-region Name,official_name_ru,Global Name,Capital,Continent,TLD,Languages,Geoname ID,CLDR display name,EDGAR
0,TPE,886,TWN,ch,Yes,158.0,925,TW,,TW,...,NaN,NaN,NaN,Taipei,AS,.tw,"zh-TW,zh,nan,hak",1668284.0,Taiwan,NaN
1,AFG,93,AFG,af,Yes,4.0,1,AF,AF,AF,...,Southern Asia,Афганистан,World,Kabul,AS,.af,"fa-AF,ps,uz-AF,tk",1149361.0,Afghanistan,B2
2,ALB,355,ALB,aa,Yes,8.0,3,AL,AB,AL,...,Southern Europe,Албания,World,Tirana,EU,.al,"sq,el",783754.0,Albania,B3
3,ALG,213,DZA,ae,Yes,12.0,4,AG,AL,DZ,...,Northern Africa,Алжир,World,Algiers,AF,.dz,ar-DZ,2589581.0,Algeria,B4
4,ASA,1-684,ASM,as,Territory of US,16.0,5,AQ,,AS,...,Polynesia,Американское Самоа,World,Pago Pago,OC,.as,"en-AS,sm,to",5880801.0,American Samoa,B5


In [12]:
country_df[['official_name_en','ISO3166-1-Alpha-3']]

,official_name_en,ISO3166-1-Alpha-3
0,NaN,TWN
1,Afghanistan,AFG
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM
5,Andorra,AND
6,Angola,AGO
7,Anguilla,AIA
8,Antarctica,ATA
9,Antigua and Barbuda,ATG


In [13]:
# Rename the columns for easier analysis in later stages - only country - code & code are required for now
country_df.rename(columns={'official_name_en': 'Name', 'ISO3166-1-Alpha-3': 'Code'}, inplace=True)

In [14]:
country_df[['Code','Name']]

,Code,Name
0,TWN,NaN
1,AFG,Afghanistan
2,ALB,Albania
3,DZA,Algeria
4,ASM,American Samoa
5,AND,Andorra
6,AGO,Angola
7,AIA,Anguilla
8,ATA,Antarctica
9,ATG,Antigua and Barbuda


In [37]:
# Create a new dataframe for the english speaking countries
eng_countries_list = [{'continent':'Asia','country':'India'},
                      {'continent':'Asia','country':'Pakistan'},
                      {'continent':'Asia','country':'Philippines'},
                      {'continent':'Asia','country':'Singapore'},
                      {'continent':'Africa','country':'Botswana'},
                      {'continent':'Africa','country':'Cameroon'},
                      {'continent':'Africa','country':'Ethiopia'},
                      {'continent':'Africa','country':'Eritrea'},
                      {'continent':'Africa','country':'The Gambia'},
                      {'continent':'Africa','country':'Ghana'},
                      {'continent':'Africa','country':'Kenya'},
                      {'continent':'Africa','country':'Lesotho'},
                      {'continent':'Africa','country':'Liberia'},
                      {'continent':'Africa','country':'Malawi'},
                      {'continent':'Africa','country':'Mauritius'},
                      {'continent':'Africa','country':'Namibia'},
                      {'continent':'Africa','country':'Nigeria'},
                      {'continent':'Africa','country':'Rwanda'},
                      {'continent':'Africa','country':'Seychelles'},
                      {'continent':'Africa','country':'Sierra Leone'},
                      {'continent':'Africa','country':'South Africa'},
                      {'continent':'Africa','country':'South Sudan'},
                      {'continent':'Africa','country':'Sudan'},
                      {'continent':'Africa','country':'Swaziland'},
                      {'continent':'Africa','country':'Tanzania'},
                      {'continent':'Africa','country':'Uganda'},
                      {'continent':'Africa','country':'Zambia'},
                      {'continent':'Africa','country':'Zimbabwe'},
                      {'continent':'Americas','country':'Antigua and Barbuda'},
                      {'continent':'Americas','country':'Bahamas'},
                      {'continent':'Americas','country':'Barbados'},
                      {'continent':'Americas','country':'Belize'},
                      {'continent':'Americas','country':'Canada'},
                      {'continent':'Americas','country':'Dominica'},
                      {'continent':'Americas','country':'Grenada'},
                      {'continent':'Americas','country':'Guyana'},
                      {'continent':'Americas','country':'Jamaica'},
                      {'continent':'Americas','country':'Saint Kitts and Nevis'},
                      {'continent':'Americas','country':'Saint Lucia'},
                      {'continent':'Americas','country':'Saint Vincent and the Grenadines'},
                      {'continent':'Americas','country':'Trinidad and Tobago'},
                      {'continent':'Americas','country':'United States of America'},
                      {'continent':'Australia/Oceania','country':'Australia'},
                      {'continent':'Australia/Oceania','country':'Fiji'},
                      {'continent':'Australia/Oceania','country':'Kiribati'},
                      {'continent':'Australia/Oceania','country':'Marshall Islands'},
                      {'continent':'Australia/Oceania','country':'Federated States of Micronesia'},
                      {'continent':'Australia/Oceania','country':'Nauru'},
                      {'continent':'Australia/Oceania','country':'New Zealand'},
                      {'continent':'Australia/Oceania','country':'Palau'},
                      {'continent':'Australia/Oceania','country':'Papua New Guinea'},
                      {'continent':'Australia/Oceania','country':'Samoa'},
                      {'continent':'Australia/Oceania','country':'Solomon Islands'},
                      {'continent':'Australia/Oceania','country':'Tonga'},
                      {'continent':'Australia/Oceania','country':'Tuvalu'},
                      {'continent':'Australia/Oceania','country':'Vanuatu'},
                      {'continent':'Europe','country':'Ireland'},
                      {'continent':'Europe','country':'Malta'},
                      {'continent':'Europe','country':'United Kingdom'}]
eng_countries_df = pd.DataFrame(eng_countries_list)

In [38]:
eng_countries_df

,continent,country
0,Asia,India
1,Asia,Pakistan
2,Asia,Philippines
3,Asia,Singapore
4,Africa,Botswana
5,Africa,Cameroon
6,Africa,Ethiopia
7,Africa,Eritrea
8,Africa,The Gambia
9,Africa,Ghana


In [22]:
# Make sure the rows/columns count matches with the .csv files
companies_df.shape

(66368, 10)

In [21]:
rounds2_df.shape

(114949, 6)

In [23]:
companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [24]:
rounds2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB


In [25]:
sector_mapping_df.shape

(688, 10)

In [26]:
sector_mapping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 10 columns):
category_list                              687 non-null object
Automotive & Sports                        688 non-null int64
Blanks                                     688 non-null int64
Cleantech / Semiconductors                 688 non-null int64
Entertainment                              688 non-null int64
Health                                     688 non-null int64
Manufacturing                              688 non-null int64
News, Search and Messaging                 688 non-null int64
Others                                     688 non-null int64
Social, Finance, Analytics, Advertising    688 non-null int64
dtypes: int64(9), object(1)
memory usage: 53.8+ KB


In [49]:
# Merge countries dataset and english speaking countries dataset
# pd.merge(df_a, df_b, on='subject_id', how='inner')
eng_country_df= pd.merge(eng_countries_df,country_df, left_on='country',right_on='Name',how='left')[['Code','country','continent']]

In [50]:
eng_country_df

,Code,country,continent
0,IND,India,Asia
1,PAK,Pakistan,Asia
2,PHL,Philippines,Asia
3,SGP,Singapore,Asia
4,BWA,Botswana,Africa
5,CMR,Cameroon,Africa
6,ETH,Ethiopia,Africa
7,ERI,Eritrea,Africa
8,NaN,The Gambia,Africa
9,GHA,Ghana,Africa


In [55]:
# update missing country codes
eng_country_df.loc[(eng_country_df.country=='United Kingdom'),'Code']='GBR'
eng_country_df.loc[(eng_country_df.country=='The Gambia'),'Code']='GMB'
eng_country_df.loc[(eng_country_df.country=='Federated States of Micronesia'),'Code']='FSM'
eng_country_df.loc[(eng_country_df.country=='Tanzania'),'Code']='TZA'
eng_country_df.loc[(eng_country_df.country=='Swaziland'),'Code']='SWZ'
# Not sure about the country codes for Federated States of Micronesia & Swaziland

In [56]:
eng_country_df

,Code,country,continent
0,IND,India,Asia
1,PAK,Pakistan,Asia
2,PHL,Philippines,Asia
3,SGP,Singapore,Asia
4,BWA,Botswana,Africa
5,CMR,Cameroon,Africa
6,ETH,Ethiopia,Africa
7,ERI,Eritrea,Africa
8,GMB,The Gambia,Africa
9,GHA,Ghana,Africa


In [59]:
# Find companies that are in English speaking countries
pd.merge(companies_df,eng_country_df, left_on='country_code',right_on='Code', how='left')

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,Code,country,continent
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,IND,India,Asia
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,USA,United States of America,Americas
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,NaN,NaN,NaN
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010,USA,United States of America,Americas
5,/Organization/01Games-Technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN,NaN,NaN,NaN
6,/Organization/0Ndine-Biomedical-Inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997,CAN,Canada,Americas
7,/Organization/0Xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,USA,United States of America,Americas
8,/Organization/1,One Inc.,http://whatis1.com,Mobile,operating,USA,CA,SF Bay Area,San Francisco,01-08-2011,USA,United States of America,Americas
9,/Organization/1-2-3-Listo,"1,2,3 Listo",http://www.123listo.com,E-Commerce,operating,CHL,12,Santiago,Las Condes,01-01-2012,NaN,NaN,NaN


In [6]:
#Find the column wise percentage null count for rounds2 dataframe 
round(100*(rounds2_df.isnull().sum()/len(rounds2_df.index)),2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

In [7]:
#Drop all the rows which have Null values for raised_amount_usd column.
rounds2_df = rounds2_df[~np.isnan(rounds2_df['raised_amount_usd'])]

In [8]:
round(100*(rounds2_df.isnull().sum()/len(rounds2_df.index)),2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         70.34
funded_at                   0.00
raised_amount_usd           0.00
dtype: float64

In [9]:
#Find the average amaount raised for each funding type
investment_type = rounds2_df.groupby('funding_round_type').raised_amount_usd.mean().sort_values(ascending = True)

In [10]:
#Find the average amaount raised for each funding type
investment_type

funding_round_type
non_equity_assistance    4.112031e+05
equity_crowdfunding      5.383682e+05
seed                     7.198180e+05
angel                    9.586945e+05
product_crowdfunding     1.363131e+06
convertible_note         1.453439e+06
grant                    4.300576e+06
venture                  1.174895e+07
debt_financing           1.704353e+07
undisclosed              1.924237e+07
private_equity           7.330859e+07
secondary_market         7.964963e+07
post_ipo_equity          8.218249e+07
post_ipo_debt            1.687046e+08
Name: raised_amount_usd, dtype: float64

In [14]:
#Converting the permlink into lower case for rounds2 dataframe
rounds2_df.apply(lambda x:x.astype(str).str.lower())

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,b,05-01-2015,10000000.0
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,nan,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,b,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,a,19-03-2008,2000000.0
6,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,nan,01-07-2014,41250.0
7,/organization/0ndine-biomedical-inc,/funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d,seed,nan,11-09-2009,43360.0
8,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,nan,21-12-2009,719491.0
9,/organization/0xdata,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,nan,22-05-2013,3000000.0
10,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,b,09-11-2015,20000000.0
11,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,nan,03-01-2013,1700000.0
